In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
sns.set(color_codes=True) # adds a nice background to the graphs
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/fetal-health-classification/fetal_health.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.head(10).style.background_gradient(cmap="RdYlBu")

In [ ]:
sns.pairplot(df)

In [ ]:
df.describe().T

In [ ]:
df.skew()

Let's check if there is any duplicates in the dataset.

In [ ]:
df[df.duplicated()]

In [ ]:
df_dup = df.drop_duplicates(subset = None , keep = 'first', inplace = False)

In [ ]:
df_dup.shape

In [ ]:
Target = df["fetal_health"]

In [ ]:
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(15, 15))
    ax = sns.heatmap(corr,mask=mask,square=True,linewidths=2.5,cmap="viridis",annot=True)

There is strong correlation between baseline value and histogram mode , histogram median and histogram mean.

Histogram number of peaks and histogram width is also having good correlation.

In [ ]:
sns.countplot(Target)
plt.show()

In [ ]:
print("Count of type 1.0 fetal health in the dataset ",len(df.loc[df["fetal_health"]==1.0]))
print("Count of type 2.0 fetal health in the dataset ",len(df.loc[df["fetal_health"]==2.0]))
print("Count of type 3.0 fetal health in the dataset ",len(df.loc[df["fetal_health"]==3.0]))

In [ ]:
updated_cols = list(df.columns)
for column in updated_cols:
    print(column," : ", len(df.loc[df[column]<0]))

In [ ]:
X = df_dup.iloc[:,:-1]
y = df_dup.iloc[:,-1]

In [ ]:
y.value_counts()

In [ ]:
scale = StandardScaler()
X = scale.fit_transform(X)
X = pd.DataFrame(X,columns=df_dup.iloc[:,:-1].columns)

Random sample Imputer for improving the class imbalance in the target column.

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ROS = RandomOverSampler(random_state=42)
X_ros, y_ros = ROS.fit_resample(X,y)
from collections import Counter
print('Resampled dataset shape %s' % Counter(y_ros))

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10, test_size = 0.2)

print('X_train', X_train.shape)
print('y_train', y_train.shape)

print('X_test', X_test.shape)
print('y_test', y_test.shape)

In [ ]:
print("{0:0.2f}% data is in training set".format((len(X_train)/len(df.index)) * 100))
print("{0:0.2f}% data is in test set".format((len(X_test)/len(y.index)) * 100))

In [ ]:
def get_train_report(model):
    
    train_pred = model.predict(X_train)
    return(classification_report(y_train, train_pred))

In [ ]:
def get_test_report(model):
    test_pred = model.predict(X_test)
    return(classification_report(y_test, test_pred))

**Decision Tree Classifier**

In [ ]:
decision_tree_classification = DecisionTreeClassifier(criterion = 'entropy', random_state = 10)
decision_tree = decision_tree_classification.fit(X_train, y_train)
from sklearn.metrics import classification_report

In [ ]:
train_report = get_train_report(decision_tree)
print(train_report)

In [ ]:
test_report = get_test_report(decision_tree)
print(test_report)

In [ ]:
dt_model = DecisionTreeClassifier(criterion = 'gini',
                                  max_depth = 5,
                                  min_samples_split = 4,
                                  max_leaf_nodes = 6,
                                  random_state = 10)

# fit the model using fit() on train data
decision_tree = dt_model.fit(X_train, y_train)
train_report = get_train_report(decision_tree)

# print the performance measures
print('Train data:\n', train_report)
test_report = get_test_report(decision_tree)

# print the performance measures
print('Test data:\n', test_report)

**Random Forest**

In [ ]:
rf_classification = RandomForestClassifier(n_estimators = 10, random_state = 10)

# use fit() to fit the model on the train set
rf_model = rf_classification.fit(X_train, y_train)

In [ ]:
train_report = get_train_report(rf_model)
print(train_report) 

In [ ]:
test_report = get_test_report(rf_model)
print(test_report) 

In [ ]:
important_features = pd.DataFrame({'Features': X_train.columns, 
                                   'Importance': rf_model.feature_importances_})

# sort the dataframe in the descending order according to the feature importance
important_features = important_features.sort_values('Importance', ascending = False)

# create a barplot to visualize the features based on their importance
sns.barplot(x = 'Importance', y = 'Features', data = important_features)

# add plot and axes labels
# set text size using 'fontsize'
plt.title('Feature Importance', fontsize = 15)
plt.xlabel('Importance', fontsize = 15)
plt.ylabel('Features', fontsize = 15)

# display the plot
plt.show()

From the above bar plot, we can see that short term variability is the most important feature in the dataset.

**K Nearest Neighbors**

In [ ]:
from sklearn.metrics import confusion_matrix,roc_curve
knn_classification = KNeighborsClassifier(n_neighbors = 3)

# fit the model using fit() on train data
knn_model = knn_classification.fit(X_train, y_train)

In [ ]:
train_report = get_train_report(knn_model)
print(train_report) 

In [ ]:
test_report = get_test_report(knn_model)
print(test_report)

In [ ]:
tuned_paramaters = {'n_neighbors': np.arange(1, 25, 2),
                   'metric': ['hamming','euclidean','manhattan','Chebyshev']}
 
# instantiate the 'KNeighborsClassifier' 
knn_classification = KNeighborsClassifier()

knn_grid = GridSearchCV(estimator = knn_classification, 
                        param_grid = tuned_paramaters, 
                        cv = 5, 
                        scoring = 'accuracy')

# fit the model on X_train and y_train using fit()
knn_grid.fit(X_train, y_train)

# get the best parameters
print('Best parameters for KNN Classifier: ', knn_grid.best_params_, '\n')

Best parameters for KNN Classifier:  {'metric': 'manhattan', 'n_neighbors': 7}

In [ ]:
from sklearn.model_selection import cross_val_score
error_rate = []

# use for loop to build a knn model for each K
for i in np.arange(1,25,2):
    
    # setup a knn classifier with k neighbors
    # use the 'euclidean' metric 
    knn = KNeighborsClassifier(i, metric = 'euclidean')
   
    # fit the model using 'cross_val_score'
    # pass the knn model as 'estimator'
    # use 5-fold cross validation
    score = cross_val_score(knn, X_train, y_train, cv = 5)
    
    # calculate the mean score
    score = score.mean()
    
    # compute error rate 
    error_rate.append(1 - score)

# plot the error_rate for different values of K 
plt.plot(range(1,25,2), error_rate)

# add plot and axes labels
# set text size using 'fontsize'
plt.title('Error Rate', fontsize = 15)
plt.xlabel('K', fontsize = 15)
plt.ylabel('Error Rate', fontsize = 15)

# set the x-axis labels
plt.xticks(np.arange(1, 25, step = 2))

# plot a vertical line across the minimum error rate
plt.axvline(x = 7, color = 'red')

# display the plot
plt.show()

We can see that the optimal value of K = 7 obtained from the GridSearchCV results in a lowest error rate.

In [ ]:
train_report = get_train_report(knn_grid)
print(train_report) 

In [ ]:
test_report = get_test_report(knn_grid)
print(test_report) 

**Gaussian Naive Bayes**

In [ ]:
gnb = GaussianNB()

# fit the model using fit() on train data
gnb_model = gnb.fit(X_train, y_train)

In [ ]:
test_report = get_test_report(gnb_model)
print(test_report) 

**Adaboost Classifier**

In [ ]:
ada_model = AdaBoostClassifier(n_estimators = 40, random_state = 10)
ada_model.fit(X_train, y_train)

In [ ]:
test_report = get_test_report(ada_model)
print(test_report)

**Gradient Boosting Classifier**

In [ ]:
gboost_model = GradientBoostingClassifier(n_estimators = 150, max_depth = 10, random_state = 10)
gboost_model.fit(X_train, y_train)

In [ ]:
test_report = get_test_report(gboost_model)
print(test_report)

**XG Boost Classifier**

In [ ]:
xgb_model = XGBClassifier(max_depth = 10, gamma = 1)
xgb_model.fit(X_train, y_train)

In [ ]:
test_report = get_test_report(xgb_model)
print(test_report)

In [ ]:
#important_features = pd.DataFrame({'Features': X_train.columns, 'Importance': xgb_model.feature_importances_})
#important_features = important_features.sort_values('Importance', ascending = False)

#sns.barplot(x = 'Importance', y = 'Features', data = important_features)
#plt.title('Feature Importance', fontsize = 15)
#plt.xlabel('Importance', fontsize = 15)
#plt.ylabel('Features', fontsize = 15)
#plt.show()

**Support Vector Machine**

In [ ]:
svc_model = SVC(kernel='poly',probability=True)
svc_model.fit(X_train,y_train)

In [ ]:
test_report = get_test_report(svc_model)
print(test_report)

Voting Classifier 

In [ ]:
clf1 = KNeighborsClassifier(n_neighbors = 7 , weights = 'distance', metric='manhattan' )
clf2 = GradientBoostingClassifier(n_estimators = 150,max_depth = 10,random_state=1)

votingclf = VotingClassifier(estimators=[('knn',clf1),('grb', clf2)],voting='hard')
votingclf = votingclf.fit(X_train,y_train)

In [ ]:
test_report = get_test_report(votingclf)
print(test_report)

We tried different algorithms for this dataset among them the boosting based algorithms i.e, XG Boost and Gradient boosting algorithms are performing best for the dataset with an accuracy of 94% on the test dataset and f1 score for XGBoost are 0.96 , 0.85 and 0.87 respectively for three different classes followed by Decision Tree Classifier without hypertuning it with an accuracy of 93% on the test data.

*PLEASE UPVOTE IF YOU LIKE THE ANALYSIS*